In [ ]:
import numpy as np
import pandas as pd
from scipy.io import *
from pylab import *
from scipy.signal import *
import scipy.signal as signal

In [ ]:
df_filtered=pd.read_csv('Merged_FilteredBands_2_100_500_10.csv')

In [ ]:
df_filtered

In [ ]:

pvalues=pd.DataFrame(columns =['Rat','Region','Theta_SlowGamma', 'Theta_MediumGamma', 'Theta_FastGamma'])
# pvalues['P_values'] = pvalues['P_values'].astype(object)


In [ ]:
def pval(rat,d, vlos, vhi, vha, vhu, region, M=False):        
    p1=0
    p2=0
    p3=0
    if M== False:
        M=len(vlos.values)
    for i in range(M):
        #get filtered bands from csv
        vlo = vlos.values[i]
        vhis = vhi.values[i]
        vhus = vhu.values[i]
        vhas = vha.values[i]
        data = d.values[i]
        Vlo = np.array([float(num) for num in vlo.split(',')[1:-1]])
        Vhi = np.array([float(num) for num in vhis.split(',')[1:-1]])
        Vhu =np.array([float(num) for num in vhus.split(',')[1:-1]])
        Vha = np.array([float(num) for num in vhas.split(',')[1:-1]])
        data = np.array([float(num) for num in data.split(',')[1:-1]])
        
        phi = angle(signal.hilbert(Vlo))  # Compute phase of low-freq signal
        amp1 = abs(signal.hilbert(Vhi))       # Compute amplitude of high-freq signal
        amp2 = abs(signal.hilbert(Vha))       # Compute amplitude of high-freq signal
        amp3 = abs(signal.hilbert(Vhu))       # Compute amplitude of high-freq signal
        p_bins = arange(-pi, pi, 0.1)
        a_mean = zeros(size(p_bins)-1)
        a2_mean = zeros(size(p_bins)-1)
        a3_mean = zeros(size(p_bins)-1)
        p_mean = zeros(size(p_bins)-1)
        
        for k in range(size(p_bins)-1):     #For each phase bin,
            pL = p_bins[k]                  #... lower phase limit,
            pR = p_bins[k+1]                #... upper phase limit.
            indices=(phi>=pL) & (phi<pR)    #Find phases falling in bin,
            a_mean[k] = mean(amp1[indices])  #... compute mean amplitude
            a2_mean[k] = mean(amp2[indices])  #... compute mean amplitude,
            a3_mean[k] = mean(amp3[indices])  #... compute mean amplitude,
            p_mean[k] = mean([pL, pR])      #... save center phase.
            
        h = max(a_mean)-min(a_mean)
        h2 = max(a2_mean)-min(a2_mean)
        h3 = max(a3_mean)-min(a3_mean)
        n_surrogates = 1000;                    #Define no. of surrogates.
        hS = zeros(n_surrogates) 
        hS2 = zeros(n_surrogates) 
        hS3 = zeros(n_surrogates) 
        N=len(data)
        #Vector to hold h results.
        for ns in range(n_surrogates):          #For each surrogate,
            ampS = amp1[randint(0,N,N)]          #Resample amplitude,
            ampS2 = amp2[randint(0,N,N)]          #Resample amplitude,
            ampS3 = amp3[randint(0,N,N)]          #Resample amplitude,
            p_bins = arange(-pi, pi, 0.1)       #Define the phase bins
            a_mean = zeros(size(p_bins)-1)      #Vector for average amps.
            a2_mean = zeros(size(p_bins)-1)
            a3_mean = zeros(size(p_bins)-1)
            p_mean = zeros(size(p_bins)-1)      #Vector for phase bins.
            
            for k in range(size(p_bins)-1):
                pL = p_bins[k]                  #... lower phase limit,
                pR = p_bins[k+1]                #... upper phase limit.
                indices=(phi>=pL) & (phi<pR)    #Find phases falling in bin,
                a_mean[k] = mean(ampS[indices]) #... compute mean amplitude,
                a2_mean[k] = mean(ampS2[indices])  #... compute mean amplitude,
                a3_mean[k] = mean(ampS3[indices])
                p_mean[k] = mean([pL, pR])      #... save center phase.
                
            hS[ns] = max(a_mean)-min(a_mean)    # Store surrogate h.
            hS2[ns] = max(a2_mean)-min(a2_mean)    # Store surrogate h.
            hS3[ns] = max(a3_mean)-min(a3_mean)    # Store surrogate h.
            
        # show()
        counts1, _, _ = hist(hS, label='surrogates slow gamma')               # Plot the histogram of hS, and save the bin counts.
        counts2, _, _ = hist(hS2, label='surrogates medium gamma')               # Plot the histogram of hS, and save the bin counts.
        counts3, _, _ = hist(hS3, label='surrogates fast gamma')               # Plot the histogram of hS, and save the bin counts.
        vlines(h, 0, max(counts1), colors='red', label='h slow gamma', lw=2)  # Plot the observed h,
        vlines(h2, 0, max(counts2), colors='green', label='h medium gamma', lw=2)  # Plot the observed h,
        vlines(h3, 0, max(counts3), colors='black', label='h fast gamma', lw=2)  # Plot the observed h,
        
#         legend(); 
#         savefig('Rat%s_Averaged_100_CFC_%s_nofast'%(rat, region))
#         show()
        p1 += sum([s > h for s in hS]) / len(hS)
        p2 += sum([s > h2 for s in hS2]) / len(hS2)
        p3 += sum([s > h3 for s in hS3]) / len(hS3)
        
        pvalues = {'Rat':rat,'Region':region,'Theta_SlowGamma':p1/M, 'Theta_MediumGamma':p2/M,'Theta_FastGamma':p3/M}
    return pvalues

In [ ]:
for rat in range(1, 10):
    if rat!=5:
        vlos = df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='PFC')),'Theta']
        vhis=df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='PFC')),'Slow_Gamma']
        vhas=df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='PFC')),'Medium_Gamma']
        vhus =df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='PFC')),'Fast_Gamma']
        data = df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='PFC')),'Rem_Data']
        pvalues=pvalues.append(pval(rat,data, vlos, vhis, vhas, vhus, 'PFC'), ignore_index=True)

In [ ]:
for rat in range(1,10):
    if rat!=5:
        vlos = df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='HPC')),'Theta']
        vhis=df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='HPC')),'Slow_Gamma']
        vhas=df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='HPC')),'Medium_Gamma']
        vhus =df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='HPC')),'Fast_Gamma']
        data = df_filtered.loc[((df_filtered.Rat== rat)&(df_filtered.Region =='HPC')),'Rem_Data']
        pvalues=pvalues.append(pval(rat,data, vlos, vhis, vhas, vhus, 'HPC'), ignore_index=True)

In [ ]:
pvalues=pvalues.sort_values('Rat')

In [ ]:
pvalues.to_csv('Data_Averaged_CFC_Random_Average_Overlapping.csv')

In [ ]:
# p_bins = arange(-pi, pi, 0.1)
# a_mean = zeros(size(p_bins)-1)
# p_mean = zeros(size(p_bins)-1)
# for k in range(size(p_bins)-1):     #For each phase bin,
#     pL = p_bins[k]                  #... lower phase limit,
#     pR = p_bins[k+1]                #... upper phase limit.
#     indices=(phi>=pL) & (phi<pR)    #Find phases falling in bin,
#     a_mean[k] = mean(amp1[indices])  #... compute mean amplitude,
#     p_mean[k] = mean([pL, pR])      #... save center phase.
# plot(p_mean, a_mean)                #Plot the phase versus amplitude,
# ylabel('High-frequency amplitude')  #... with axes labeled.
# xlabel('Low-frequency phase');


In [ ]:
# h = max(a_mean)-min(a_mean)
# n_surrogates = 1000;                    #Define no. of surrogates.
# hS = zeros(n_surrogates) 
# N=len(data)
# #Vector to hold h results.
# for ns in range(n_surrogates):          #For each surrogate,
#     ampS = amp1[randint(0,N,N)]          #Resample amplitude,
#     p_bins = arange(-pi, pi, 0.1)       #Define the phase bins
#     a_mean = zeros(size(p_bins)-1)      #Vector for average amps.
#     p_mean = zeros(size(p_bins)-1)      #Vector for phase bins.
#     for k in range(size(p_bins)-1):
#         pL = p_bins[k]                  #... lower phase limit,
#         pR = p_bins[k+1]                #... upper phase limit.
#         indices=(phi>=pL) & (phi<pR)    #Find phases falling in bin,
#         a_mean[k] = mean(ampS[indices]) #... compute mean amplitude,
#         p_mean[k] = mean([pL, pR])      #... save center phase.
#     hS[ns] = max(a_mean)-min(a_mean)    # Store surrogate h.
# show()
# counts, _, _ = hist(hS, label='surrogates')               # Plot the histogram of hS, and save the bin counts.
# vlines(h, 0, max(counts), colors='red', label='h', lw=2)  # Plot the observed h,
# legend(); 
# show()
# p = sum([s > h for s in hS]) / len(hS)
# print(p)